In [3]:
%cd ../..

/home/matheus/Desktop/Itens/Projetos/paper - llm embeddings to classification - marcacini - matheus


In [4]:
import os
import pandas as pd
import json

# Configura o Pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

def load_results_to_dataframe(base_path: str) -> pd.DataFrame:
    """
    Load results from JSON files into a pandas DataFrame.

    This function traverses the directory structure under the given base path, 
    finds the results.json files, calculates the average for list values, 
    and compiles the results into a DataFrame with additional metadata 
    such as dataset name, model type, and model name.

    Parameters:
    - base_path (str): The base directory path containing the results.

    Returns:
    - pd.DataFrame: A DataFrame containing the consolidated results.
    """
    results = []
    
    # Traverse the directory structure
    for dataset_name in os.listdir(base_path):
        dataset_path = os.path.join(base_path, dataset_name)
        
        if os.path.isdir(dataset_path):
            for model_type in os.listdir(dataset_path):
                model_type_path = os.path.join(dataset_path, model_type)
                
                if os.path.isdir(model_type_path):
                    for model_name in os.listdir(model_type_path):
                        model_name_path = os.path.join(model_type_path, model_name)
                        json_file_path = os.path.join(model_name_path, 'results.json')
                        
                        # Check if results.json file exists
                        if os.path.isfile(json_file_path):
                            with open(json_file_path, 'r') as json_file:
                                result_data = json.load(json_file)
                                
                                # Calculate the mean for each list
                                for key in result_data.keys():
                                    if isinstance(result_data[key], list):
                                        result_data[key] = sum(result_data[key]) / len(result_data[key])
                                
                                # Add metadata to the result data
                                result_data['dataset_name'] = dataset_name
                                result_data['model_type'] = model_type
                                result_data['model_name'] = model_name
                                results.append(result_data)

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    # Specify the order of the columns, putting 'dataset_name', 'model_type', and 'model_name' first
    columns_first = ['dataset_name', 'model_type', 'model_name']
    column_order = columns_first + [col for col in results_df.columns if col not in columns_first]
    results_df = results_df[column_order]
    
    return results_df

In [5]:
# Usage
base_path = 'results' 
results_df = load_results_to_dataframe(base_path)

print("DataFrame of Results:")
display(results_df)

# Save the DataFrame to a CSV file
results_df.to_csv(f'{base_path}_result_resume.csv', index=False)  # Include index=False to avoid saving the index as a column

DataFrame of Results:


,dataset_name,model_type,model_name,fit_time,score_time,test_accuracy,train_accuracy,test_precision,train_precision,test_recall,train_recall,test_f1_score,train_f1_score,embedding_generation_time
0,CSTR.csv,llama_cpp,lmstudio-community_Llama-3.2-3B-Instruct-GGUF,0.012874,0.383841,0.872994,0.900499,0.894219,0.916084,0.893410,0.921155,0.889607,0.917097,69.661079
1,CSTR.csv,bert,sentence-transformers_all-mpnet-base-v1,0.003274,0.094472,0.879774,0.904686,0.896697,0.907133,0.903256,0.920498,0.896097,0.912791,11.326948
